In [1]:
# モジュールのインポート
!apt-get update
!apt install chromium-chromedriver
!cp /usr/lib/chromium-browser/chromedriver /usr/bin
!pip install selenium

Get:1 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease [15.9 kB]
Hit:2 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:3 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Get:4 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu bionic InRelease [21.3 kB]
Get:5 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Get:6 http://archive.ubuntu.com/ubuntu bionic-backports InRelease [74.6 kB]
Get:7 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Ign:8 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Ign:9 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Get:10 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release [697 B]
Hit:11 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Get:12 https://developer.download.nvidia.com/compute/cuda/repo

In [ ]:
#webdriver, optionsのインポート
from selenium import webdriver
from selenium.webdriver.chrome.options import Options

In [ ]:
options = webdriver.ChromeOptions()
# GUIを持たない Headless モードに設定。新しいブラウザを別途使わないようにする。
options.add_argument('--headless')
options.add_argument('--no-sandbox')
options.add_argument('--disable-dev-shm-usage')
browser = webdriver.Chrome('chromedriver',options=options)

In [ ]:
# Webサイトにアクセス
url = 'http://4travel.jp/domestic/area/kinki/kyoto/kankospot'
browser.get(url)

In [ ]:
browser.quit()

###観光地名を1件取得

In [ ]:
title_elem = browser.find_element_by_class_name('u_title')
title_elem.text

'1\n清水寺 寺・神社 東山・祇園・北白川'

In [ ]:
elem = title_elem.find_element_by_tag_name('h2')
elem.text

'清水寺'

In [ ]:
elem.find_element_by_tag_name('a').text

'清水寺'

In [ ]:
url = elem.find_element_by_tag_name('a').get_attribute('href')
url

'https://4travel.jp/dm_shisetsu/10002108'

###観光地名を複数取得

In [ ]:
title_elems = browser.find_elements_by_class_name('u_title')
len(title_elems)

20

In [ ]:
title_elems[0].text

'1\n清水寺 寺・神社 東山・祇園・北白川'

In [ ]:
#観光地名だけを取得
title_elem = title_elems[0]
elem = title_elem.find_element_by_tag_name('h2')
elem.text

'清水寺'

###演習問題１
20個の観光地名をplaces[]というリストに格納

In [ ]:
title_elems = browser.find_elements_by_class_name('u_title')

places = []
for title_elem in title_elems:
  elem = title_elem.find_element_by_tag_name('h2')
  places.append(elem.text)

In [ ]:
places

['清水寺',
 '伏見稲荷大社',
 '鹿苑寺(金閣寺)',
 '平等院',
 '八坂神社',
 '渡月橋',
 '京都タワー',
 '南禅寺',
 '錦市場',
 '天龍寺',
 '嵯峨野 竹林の道',
 '平安神宮',
 '東福寺',
 '東寺(教王護国寺)',
 '元離宮二条城',
 '下鴨神社(賀茂御祖神社)',
 '銀閣寺(慈照寺)',
 '三十三間堂(蓮華王院)',
 '鴨川',
 '高台寺']

###総合評価(点数)でも同じことをやってみよう。

In [ ]:
#総合評価の取得。まずは1件のみ。
evaluate_elem = browser.find_element_by_class_name('ranking_evaluateWrapper')
evaluate_elem.text

'4.53 評価詳細'

In [ ]:
#点数のみを取得
evaluate = evaluate_elem.find_element_by_tag_name('span')
evaluate.text

'4.53'

In [ ]:
#複数取得
evaluate_elems = browser.find_elements_by_class_name('ranking_evaluateWrapper')
len(evaluate_elems)

20

In [ ]:
evaluate_elems[0].text

'4.53 評価詳細'

In [ ]:
#0番目の点数だけを取得
evaluate_elem = evaluate_elems[0]
elem = evaluate_elem.find_element_by_tag_name('span')
elem.text

'4.53'

In [ ]:
evaluate_elems = browser.find_elements_by_class_name('ranking_evaluateWrapper')

evaluates = []
for evaluate_elem in evaluate_elems:
  elem = evaluate_elem.find_element_by_tag_name('span')
  evaluates.append(elem.text)

In [ ]:
evaluates

['4.53',
 '4.53',
 '4.42',
 '4.40',
 '4.34',
 '4.34',
 '4.32',
 '4.30',
 '4.30',
 '4.29',
 '4.27',
 '4.26',
 '4.26',
 '4.24',
 '4.24',
 '4.23',
 '4.23',
 '4.22',
 '4.21',
 '4.21']

ちなみに、いきなり「evaluateNumber」のclass指定をしてしまうとうまく行かない。  
これは、そのclassで指定されているものが他にもあるから。  
特定の情報を取得したい場合は、その情報が所属するdivをまずは指定して、そのdivの中で、その情報をさらに指定するような形が望ましい。  
つまり、先ほどの観光地名でやった流れに沿ってやるのが良い。  
- 自分がやったのを後で修正しよう。spanで取得するのは気持ち悪い。

In [ ]:
# アクセスのタイミングを開ける
from time import sleep

# 複数個に適用
block_elems = browser.find_elements_by_class_name('u_areaListRankingBox')
# block_elem = block_elems[0]  # <- point(for文に置き換える前のポイント)

access_ranks = []
for block_elem in block_elems:
    elems = block_elem.find_element_by_class_name('evaluateNumber')
    # print(elem.text) # 確認用
    access_ranks.append(elems.text)
    print('処理中')
    # アクセスの間隔を５秒開ける
    sleep(5)

処理中
処理中
処理中
処理中
処理中
処理中
処理中
処理中
処理中
処理中
処理中
処理中
処理中
処理中
処理中
処理中
処理中
処理中
処理中
処理中


In [ ]:
access_ranks[0], type(access_ranks[0])

('4.53', str)

In [ ]:
# 選択肢1
com_ranks = []
for rank in access_ranks:
    com_ranks.append( float(rank) )

com_ranks, type(com_ranks[0])

([4.53,
  4.53,
  4.42,
  4.4,
  4.34,
  4.34,
  4.32,
  4.3,
  4.3,
  4.29,
  4.27,
  4.26,
  4.26,
  4.24,
  4.24,
  4.23,
  4.23,
  4.22,
  4.21,
  4.21],
 float)

###データの整形

In [ ]:
import pandas as pd
df = pd.DataFrame()

In [ ]:
df['観光地'] = places

In [ ]:
df['総合評価'] = com_ranks

In [ ]:
df

,観光地,総合評価
0,清水寺,4.53
1,伏見稲荷大社,4.53
2,鹿苑寺(金閣寺),4.42
3,平等院,4.40
4,八坂神社,4.34
5,渡月橋,4.34
6,京都タワー,4.32
7,南禅寺,4.30
8,錦市場,4.30
9,天龍寺,4.29


###条件抽出

In [ ]:
df[df['総合評価'] > 4.30]

,観光地,総合評価
0,清水寺,4.53
1,伏見稲荷大社,4.53
2,鹿苑寺(金閣寺),4.42
3,平等院,4.40
4,八坂神社,4.34
5,渡月橋,4.34
6,京都タワー,4.32


In [ ]:
df[ (df['総合評価'] >= 4.30) & (df['総合評価'] <4.50)]

,観光地,総合評価
2,鹿苑寺(金閣寺),4.42
3,平等院,4.40
4,八坂神社,4.34
5,渡月橋,4.34
6,京都タワー,4.32
7,南禅寺,4.30
8,錦市場,4.30


###CSVファイルへの出力

In [ ]:
df_csv = df[ (df['総合評価'] >= 4.30) & (df['総合評価'] <4.50)]

In [ ]:
df_csv.to_csv('celenium.csv', index=False, encoding='shift-jis')

In [ ]:
browser.quit()

###BeautifulSoup4

In [ ]:
!pip install beautifulsoup4

In [ ]:
from bs4 import BeautifulSoup as bs4

In [ ]:
import requests

In [ ]:
url = 'https://4travel.jp/domestic/area/kinki/kyoto/kankospot'
res = requests.get(url)

In [ ]:
soup = bs4(res.text, 'html.parser')
print(soup.prettify)

<bound method Tag.prettify of <!DOCTYPE html>

<html class="mode-default" lang="ja" xmlns="http://www.w3.org/1999/xhtml" xmlns:fb="http://www.facebook.com/2008/fbml">
<head>
<meta charset="utf-8"/>
<title>京都のおすすめ観光スポット クチコミ人気ランキングTOP40【フォートラベル】</title>
<link href="https://4travel.jp/domestic/area/kinki/kyoto/kankospot" rel="canonical"/>
<meta content="ba230d5e0e36a402" name="y_key">
<meta content="37331208E5607E16D824561A9C77272D" name="msvalidate.01"/>
<meta content="noimage" name="mixi-check-robots"/>
<meta content="236780919753578" property="fb:app_id"/>
<link href="https://4travel.jp/favicon.ico" rel="shortcut icon"/>
<meta content="京都で人気の観光スポットをランキング形式でご紹介！旅行者のクチコミをもとに京都の観光スポットが探せます。最新の観光スポット情報は日本最大級の旅行クチコミサイト フォートラベルでチェック！" name="description"/>
<meta content="京都,観光,スポット,人気,ランキング,クチコミ,割引チケット,クーポン" name="keywords"/>
<meta content="https://cdn.4travel.jp/img/webapp/banner/ogp/ogp_logo_travel_photo.png" property="og:image"/>
<link href="https://4travel.jp/domestic/area/kinki/kyoto/ka

In [ ]:
contents = soup.select('.u_title')
print(contents)

[<div class="u_title">
<span class="u_iconRanking is_iconNo1">1</span>
<h2>
<a href="https://4travel.jp/dm_shisetsu/10002108" target="_blank">清水寺</a>
</h2>
<a href="https://4travel.jp/domestic/area/kinki/kyoto/temple">
<small>寺・神社</small>
</a>
<span class="areaTextItem"><a href="https://4travel.jp/dm_area_kuchoson-higashiyama.html">東山・祇園・北白川</a></span>
</div>, <div class="u_title">
<span class="u_iconRanking is_iconNo2">2</span>
<h2>
<a href="https://4travel.jp/dm_shisetsu/10002206" target="_blank">伏見稲荷大社</a>
</h2>
<a href="https://4travel.jp/domestic/area/kinki/kyoto/temple">
<small>寺・神社</small>
</a>
<span class="areaTextItem"><a href="https://4travel.jp/dm_area_kuchoson-fushimi.html">伏見</a></span>
</div>, <div class="u_title">
<span class="u_iconRanking is_iconNo3">3</span>
<h2>
<a href="https://4travel.jp/dm_shisetsu/10002066" target="_blank">鹿苑寺(金閣寺)</a>
</h2>
<a href="https://4travel.jp/domestic/area/kinki/kyoto/temple">
<small>寺・神社</small>
</a>
<span class="areaTextItem"><a href=

In [ ]:
contents = soup.select('.u_title > h2 > a')
contents[0].string

'清水寺'

###演習３

In [ ]:
contents = soup.select('.u_title > h2 > a')

places = []
for content in contents:
  places.append(content.string)

In [ ]:
places

['清水寺',
 '伏見稲荷大社',
 '鹿苑寺(金閣寺)',
 '平等院',
 '八坂神社',
 '渡月橋',
 '京都タワー',
 '南禅寺',
 '錦市場',
 '天龍寺',
 '嵯峨野 竹林の道',
 '平安神宮',
 '東福寺',
 '東寺(教王護国寺)',
 '元離宮二条城',
 '下鴨神社(賀茂御祖神社)',
 '銀閣寺(慈照寺)',
 '三十三間堂(蓮華王院)',
 '高台寺',
 '鴨川']

###演習４

In [ ]:
evaluates = soup.select('.is_rank > span')

com_ranks = []
for evaluate in evaluates:
  com_ranks.append(evaluate.string)

In [ ]:
com_ranks

['4.53',
 '4.52',
 '4.42',
 '4.40',
 '4.34',
 '4.34',
 '4.32',
 '4.31',
 '4.30',
 '4.29',
 '4.27',
 '4.27',
 '4.26',
 '4.24',
 '4.24',
 '4.23',
 '4.23',
 '4.22',
 '4.21',
 '4.21']

In [ ]:
com_ranks[2]

'4.42'

In [ ]:
import pandas as pd


In [ ]:
df = pd.DataFrame({
    '観光地': places,
    '総合評価': com_ranks,
})

In [ ]:
df

,観光地,総合評価
0,清水寺,4.53
1,伏見稲荷大社,4.52
2,鹿苑寺(金閣寺),4.42
3,平等院,4.40
4,八坂神社,4.34
5,渡月橋,4.34
6,京都タワー,4.32
7,南禅寺,4.31
8,錦市場,4.30
9,天龍寺,4.29


In [ ]:
df.to_csv('selenium2.csv', index=False, encoding='shift-jis')

In [ ]:
df.to_csv('selenium3.csv', index=False, encoding='Shift-JIS')

In [ ]:
df.to_csv('selenium4.csv', index=False, encoding='UTF-8')

In [ ]:
df.to_csv('selenium5.csv', index=False, encoding='Shift-JIS')